In [18]:
import pandas as pd
import numpy as np

df = pd.read_csv('hair_type_data.csv')
df.head()

,image_name,img_b64,hair_type
0,02dac897d1dec9ba8c057a11d041ada8--layered-natu...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIB...,curly
1,090617-header-mobile.jpg,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIB...,curly
2,0b7aa42d28a71654681bfd13b569ce32.jpg,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIB...,curly
3,0dba94f77ba91fe4c546048f7476c0ed--curly-hair-c...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIB...,curly
4,120047603-selfie-photo-of-curly-haired-caucasi...,b'/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAIBAQEBAQIB...,curly


In [19]:
from PIL import Image
from io import BytesIO
import base64

#decode image from base64 and store as flattened list
df['img_list'] = df['img_b64'].apply(lambda x : np.array(Image.open(BytesIO(base64.b64decode(x[2:])))).reshape(-1).tolist())

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 659 entries, 0 to 658
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image_name  659 non-null    object
 1   img_b64     659 non-null    object
 2   hair_type   659 non-null    object
 3   img_list    659 non-null    object
dtypes: object(4)
memory usage: 20.7+ KB


In [21]:
#find max len of list
list_lens = []
for iter, row in df.iterrows():
    list_lens.append(len(row['img_list']))
max_to_pad = max(list_lens)

#apply padding to all lists
df['img_list'] = df['img_list'].apply(lambda x : np.pad(x, (0, max_to_pad - len(x)), mode='constant'))

In [22]:
#get columns and explode into new dataframe
list_len = max_to_pad
cols = []
for i in range(list_len):
    cols.append('img_feature' + str(i))

df2 = pd.DataFrame(df.img_list.tolist(), columns=cols, index=df.index)

In [24]:
#assign features and train/test split
feature_columns = df2.columns[:-1]
features = df2[feature_columns]
target = df['hair_type']

In [25]:
# Define a normalization function that scales values to [0, 1]
def normalize_column(column):
    min_val = column.min()
    max_val = column.max()
    return (column - min_val) / (max_val - min_val)

# Apply the normalization function to each column of features
features = features.apply(normalize_column, axis=1)

In [26]:
features

,img_feature0,img_feature1,img_feature2,img_feature3,img_feature4,img_feature5,img_feature6,img_feature7,img_feature8,img_feature9,...,img_feature67489,img_feature67490,img_feature67491,img_feature67492,img_feature67493,img_feature67494,img_feature67495,img_feature67496,img_feature67497,img_feature67498
0,0.547718,0.535270,0.497925,0.556017,0.543568,0.506224,0.560166,0.543568,0.506224,0.560166,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.952941,0.956863,0.972549,0.956863,0.960784,0.976471,0.956863,0.960784,0.976471,0.952941,...,0.137255,0.145098,0.180392,0.101961,0.109804,0.231373,0.145098,0.156863,0.203922,0.117647
2,0.466667,0.368627,0.282353,0.415686,0.329412,0.247059,0.552941,0.470588,0.396078,0.564706,...,0.337255,0.290196,0.274510,0.258824,0.215686,0.298039,0.298039,0.258824,0.298039,0.298039
3,0.521569,0.474510,0.380392,0.513725,0.466667,0.372549,0.513725,0.466667,0.372549,0.513725,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.882353,0.905882,1.000000,0.866667,0.913725,1.000000,0.682353,0.760784,0.854902,0.454902,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654,0.529412,0.474510,0.372549,0.525490,0.470588,0.364706,0.529412,0.478431,0.352941,0.533333,...,0.298039,0.200000,0.400000,0.298039,0.200000,0.396078,0.294118,0.196078,0.392157,0.298039
655,0.149020,0.137255,0.062745,0.164706,0.145098,0.070588,0.156863,0.129412,0.058824,0.145098,...,0.356863,0.215686,0.431373,0.360784,0.219608,0.423529,0.360784,0.219608,0.419608,0.356863
656,0.149020,0.137255,0.062745,0.164706,0.145098,0.070588,0.156863,0.129412,0.058824,0.145098,...,0.356863,0.215686,0.431373,0.360784,0.219608,0.423529,0.360784,0.219608,0.419608,0.356863
657,0.142857,0.115079,0.091270,0.170635,0.142857,0.119048,0.222222,0.186508,0.158730,0.238095,...,0.829365,0.773810,0.876984,0.829365,0.773810,0.876984,0.829365,0.773810,0.884921,0.829365


In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=5)

In [28]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [29]:
#Train classifiers
clf_decision_tree = DecisionTreeClassifier(max_depth=200, random_state=0)
clf_decision_tree.fit(X_train, y_train)
clf_random_forest = RandomForestClassifier(max_depth=200, random_state=0)
clf_random_forest.fit(X_train, y_train)
clf_gaussian_nb = GaussianNB()
clf_gaussian_nb.fit(X_train, y_train)
clf_KNN = KNeighborsClassifier(n_neighbors=10, algorithm='ball_tree')
clf_KNN.fit(X_train, y_train)
clf_SVC = SVC(gamma='auto')
clf_SVC.fit(X_train, y_train)
clf_ABC = AdaBoostClassifier(n_estimators=50, learning_rate=1)
clf_ABC.fit(X_train, y_train)

AdaBoostClassifier(learning_rate=1)

<hr>


A function to evaluate our model, thanks to Rafi Atha, his article is at: https://medium.com/analytics-vidhya/building-classification-model-with-python-9bdfc13faa4b

In [30]:
def evaluate_model(model, x_test, y_test):
    from sklearn import metrics

    # Predict Test Data 
    y_pred = model.predict(x_test)

    # Calculate accuracy, precision, recall, f1-score, and kappa score
    acc = metrics.accuracy_score(y_test, y_pred)
    prec = metrics.precision_score(y_test, y_pred, average='micro')
    rec = metrics.recall_score(y_test, y_pred, average='micro')
    f1 = metrics.f1_score(y_test, y_pred, average='micro')
    kappa = metrics.cohen_kappa_score(y_test, y_pred)

    return {'accuracy': acc, 'precision': prec, 'recall score': rec, 'f1 score': f1, 'kappa': kappa}

<hr>

In [31]:
# Evaluate Model
rf_eval_dec_tree = evaluate_model(clf_decision_tree, X_test, y_test)
rf_eval_dec_tree['classifier'] = 'Decision Tree'
rf_eval_rand_forest = evaluate_model(clf_random_forest, X_test, y_test)
rf_eval_rand_forest['classifier'] = 'Random Forest'
rf_eval_gaussian_nb = evaluate_model(clf_gaussian_nb, X_test, y_test)
rf_eval_gaussian_nb['classifier'] = 'Gaussian NB'
rf_eval_knn = evaluate_model(clf_KNN, X_test, y_test)
rf_eval_knn['classifier'] = 'KNeighbor'
rf_eval_ada = evaluate_model(clf_ABC, X_test, y_test)
rf_eval_ada['classifier'] = 'Ada Boost'

eval = pd.DataFrame([rf_eval_dec_tree, rf_eval_rand_forest, rf_eval_gaussian_nb,
                     rf_eval_knn, rf_eval_ada], columns=['classifier', 'accuracy', 'precision', 'recall score', 'f1 score', 'kappa'])
eval

,classifier,accuracy,precision,recall score,f1 score,kappa
0,Decision Tree,0.533333,0.533333,0.533333,0.533333,0.297057
1,Random Forest,0.581818,0.581818,0.581818,0.581818,0.370229
2,Gaussian NB,0.460606,0.460606,0.460606,0.460606,0.201815
3,KNeighbor,0.478788,0.478788,0.478788,0.478788,0.208236
4,Ada Boost,0.472727,0.472727,0.472727,0.472727,0.189990


In [32]:
#save model locally
import pickle
with open('model_hair.pk1', 'wb') as file:  
    pickle.dump(clf_random_forest, file)